In [6]:
"""
PDF章节切割工具
根据"专项训练"关键词识别章节并切割PDF
"""

import re
from pathlib import Path
import PyPDF2
import pdfplumber

def extract_text_from_page(pdf_path, page_num):
    """
    提取指定页面的文本内容
    
    Args:
        pdf_path: PDF文件路径
        page_num: 页码（从0开始）
    
    Returns:
        页面文本内容
    """
    with pdfplumber.open(pdf_path) as pdf:
        if page_num < len(pdf.pages):
            page = pdf.pages[page_num]
            return page.extract_text() or ""
    return ""

def find_chapter_pages(pdf_path):
    """
    查找所有章节的起始页
    
    Args:
        pdf_path: PDF文件路径
    
    Returns:
        章节信息列表，每项包含 (起始页码, 章节标题)
    """
    chapters = []
    pattern = re.compile(r'^专项训练[一二三四五六七八九十\d]+\s+(.+)', re.MULTILINE)
    
    with pdfplumber.open(pdf_path) as pdf:
        total_pages = len(pdf.pages)
        
        for page_num in range(total_pages):
            text = pdf.pages[page_num].extract_text()
            if not text:
                continue
            
            # 获取第一行文本
            lines = text.strip().split('\n')
            if lines:
                first_line = lines[0].strip()
                match = pattern.match(first_line)
                
                if match:
                    chapter_title = first_line
                    chapters.append((page_num, chapter_title))
                    print(f"找到章节: 第{page_num + 1}页 - {chapter_title}")
    
    return chapters

def split_pdf_by_chapters(input_pdf, output_dir=None):
    """
    按章节切割PDF文件
    
    Args:
        input_pdf: 输入PDF文件路径
        output_dir: 输出目录（默认为与输入文件同目录）
    """
    input_path = Path(input_pdf)
    
    if not input_path.exists():
        print(f"错误: 文件不存在 - {input_pdf}")
        return
    
    # 设置输出目录
    if output_dir is None:
        output_dir = input_path.parent / f"{input_path.stem}_chapters"
    else:
        output_dir = Path(output_dir)
    
    output_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"正在分析PDF文件: {input_pdf}")
    print(f"输出目录: {output_dir}")
    print("-" * 60)
    
    # 查找所有章节
    chapters = find_chapter_pages(str(input_path))
    
    if not chapters:
        print("未找到任何章节标记")
        return
    
    print(f"\n共找到 {len(chapters)} 个章节")
    print("-" * 60)
    
    # 读取PDF
    with open(input_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        total_pages = len(pdf_reader.pages)
        
        # 切割每个章节
        for i, (start_page, chapter_title) in enumerate(chapters):
            # 确定结束页
            if i < len(chapters) - 1:
                end_page = chapters[i + 1][0] - 1  # 下一章节前一页
            else:
                end_page = total_pages - 1  # 最后一页
            
            # 创建新PDF
            pdf_writer = PyPDF2.PdfWriter()
            
            # 添加页面
            for page_num in range(start_page, end_page + 1):
                pdf_writer.add_page(pdf_reader.pages[page_num])
            
            # 生成安全的文件名
            safe_title = str(i+1) + ". " + re.sub(r'[\\/:*?"<>|]', '_', chapter_title)
            output_file = output_dir / f"{safe_title}.pdf"
            
            # 写入文件
            with open(output_file, 'wb') as output:
                pdf_writer.write(output)
            
            print(f"✓ 已生成: {output_file.name}")
            print(f"  页面范围: {start_page + 1} - {end_page + 1} (共{end_page - start_page + 1}页)")
    
    print("-" * 60)
    print(f"完成! 共生成 {len(chapters)} 个PDF文件")
    print(f"保存位置: {output_dir}")

In [8]:
# 获取输入文件
input_pdf = "2026刷题班讲义答案.pdf"
output_dir = None

print("\n开始处理...\n")

try:
  split_pdf_by_chapters(input_pdf, output_dir)
except Exception as e:
  print(f"\n错误: {e}")
  import traceback
  traceback.print_exc()


开始处理...

正在分析PDF文件: 2026刷题班讲义答案.pdf
输出目录: 2026刷题班讲义答案_chapters
------------------------------------------------------------
找到章节: 第2页 - 专项训练一 归纳概括
找到章节: 第8页 - 专项训练二 综合分析
找到章节: 第16页 - 专项训练三 提出对策
找到章节: 第25页 - 专项训练四 公文写作
找到章节: 第35页 - 专项训练五 应用文写作
找到章节: 第45页 - 专项训练六 文章写作（A 类）
找到章节: 第55页 - 专项训练七 文章写作（B 类）

共找到 7 个章节
------------------------------------------------------------
✓ 已生成: 1. 专项训练一 归纳概括.pdf
  页面范围: 2 - 7 (共6页)
✓ 已生成: 2. 专项训练二 综合分析.pdf
  页面范围: 8 - 15 (共8页)
✓ 已生成: 3. 专项训练三 提出对策.pdf
  页面范围: 16 - 24 (共9页)
✓ 已生成: 4. 专项训练四 公文写作.pdf
  页面范围: 25 - 34 (共10页)
✓ 已生成: 5. 专项训练五 应用文写作.pdf
  页面范围: 35 - 44 (共10页)
✓ 已生成: 6. 专项训练六 文章写作（A 类）.pdf
  页面范围: 45 - 54 (共10页)
✓ 已生成: 7. 专项训练七 文章写作（B 类）.pdf
  页面范围: 55 - 64 (共10页)
------------------------------------------------------------
完成! 共生成 7 个PDF文件
保存位置: 2026刷题班讲义答案_chapters
